In [1]:
import pandas as pd
df = pd.read_csv("parlaspeech.csv")
df.columns

Index(['file', 'start', 'end', 'words', 'word_start_times', 'word_end_times',
       'asr', 'true_file', 'reconame', 'longname', 'yt_hash', 'hashname',
       'key', 'was_in_samples', 'human_transcript', 'asr_transcript', 'sim',
       'split'],
      dtype='object')

In [2]:
df_pruned = df.loc[:, ['file', 'start', 'end',
     'true_file', 'reconame', 'yt_hash', 'hashname',
     'was_in_samples', 'human_transcript', 'asr_transcript', 'sim',]]

In [3]:
df_pruned.to_csv("parlaspeech_pruned.csv", index=False)

In [4]:
df_pruned.to_json("parlaspeech_pruned.json", orient="records", lines=True)

In [10]:
estimated_size = df_pruned.shape[0]/320174 * 321
estimated_size

448.6965899791988

In [11]:
c_to_drop = df_pruned.was_in_samples | df_pruned.sim<0.8
df_pruned = df_pruned.drop(index=df_pruned.index[c_to_drop])
available = 232141524 
corpus_size = 450725396

train_fraction = available/corpus_size
train_fraction

0.5150398137317295

Zaokrožim na 0.5

In [12]:
keep_df = df_pruned.sample(frac=0.5, random_state=42)
keep_df.to_csv("keep_pruned.csv", index=False)

In [14]:
from concurrent.futures import ProcessPoolExecutor
from shutil import copyfile
import os
def copy(file):
    if os.path.exists(os.path.join("/home/rupnik/macocu/task8/transfer", file)):
        return

    copyfile(os.path.join("/home/rupnik/macocu/task8/data_16000", file),
            os.path.join("/home/rupnik/macocu/task8/transfer", file)
            )
with ProcessPoolExecutor(max_workers=100) as executor:
    executor.map(copy, keep_df.hashname.values)